In [9]:
# 기본 import
import os, glob, json
import numpy as np
import cv2

from pathlib import Path
from tqdm import tqdm

In [16]:
# === 입력/출력 설정 ===
INPUT_PATH   = "test_0"              # 폴더 경로 또는 단일 JPG 파일 경로
OUTPUT_DIR   = "test_output"       # 보정 결과가 저장될 폴더
SIDE_BY_SIDE = True                # 원본/보정 비교 이미지 추가 저장할지

# === 보정 모드 선택 ===
#   1) 표준/어안 보정 파일(.npz) 사용
USE_NPZ      = True
CAL_NPZ_PATH = "pi_cam_calib.npz"  # standard: pi_cam_calib.npz / fisheye: pi_cam_fisheye.npz

#   2) 수동 JSON 파라미터 사용(간이 튜닝 후 생성됨)
USE_MANUAL_JSON = False
MANUAL_JSON_PATH = "manual_calib.json"

# === 크롭(화각) 트레이드오프 ===
# standard: alpha(0.0=더 많이 크롭, 직선 최대 ~ 1.0=크롭 적음)
ALPHA   = 0.0
# fisheye: balance(0.0=직선 최대 ~ 1.0=크롭 적음/화각 크게)
BALANCE = 0.3


In [17]:
IMG_EXTS = ("*.jpg","*.JPG","*.jpeg","*.JPEG")

def list_images(path_str: str):
    """폴더면 JPG 전체, 파일이면 그 파일만 리스트로 반환"""
    p = Path(path_str)
    if p.is_dir():
        files = []
        for ext in IMG_EXTS:
            files += sorted(glob.glob(str(p / ext)))
        return files
    elif p.is_file():
        return [str(p)]
    else:
        raise FileNotFoundError(f"Input not found: {p}")

def scale_K(K, w0, h0, W, H):
    """해상도 변경 시 내부행렬 스케일링"""
    sx, sy = W / float(w0), H / float(h0)
    Ks = K.copy()
    Ks[0,0] *= sx; Ks[1,1] *= sy
    Ks[0,2] *= sx; Ks[1,2] *= sy
    return Ks

def build_maps_standard(K, dist, size, alpha=0.0):
    W, H = size
    newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (W,H), alpha)
    map1, map2 = cv2.initUndistortRectifyMap(K, dist, None, newK, (W,H), cv2.CV_16SC2)
    return map1, map2

def build_maps_fisheye(K, D, size, balance=0.0):
    W, H = size
    newK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, (W,H), np.eye(3), balance=balance)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), newK, (W,H), cv2.CV_16SC2)
    return map1, map2

def save_side_by_side(src, und, out_path):
    h = max(src.shape[0], und.shape[0])
    w = src.shape[1] + und.shape[1]
    canvas = np.zeros((h, w, 3), dtype=src.dtype)
    canvas[:src.shape[0], :src.shape[1]] = src
    canvas[:und.shape[0], src.shape[1]:src.shape[1]+und.shape[1]] = und
    cv2.imwrite(out_path, canvas)


In [18]:
def batch_undistort_with_npz(files, out_dir, cal_path, alpha=0.0, balance=0.0, side_by_side=False):
    cal = np.load(cal_path)
    use_fisheye = ('D' in cal and 'K' in cal and 'dist' not in cal)
    K = cal['K']
    if use_fisheye:
        D = cal['D']
        w0 = int(cal['w']) if 'w' in cal else None
        h0 = int(cal['h']) if 'h' in cal else None
    else:
        dist = cal['dist']
        w0 = int(cal['w']) if 'w' in cal else None
        h0 = int(cal['h']) if 'h' in cal else None

    os.makedirs(out_dir, exist_ok=True)
    cache = {}  # (W,H) -> (map1,map2)

    for f in tqdm(files, desc="Undistorting (npz)"):
        img = cv2.imread(f)
        if img is None:
            continue
        H, W = img.shape[:2]
        key = (W, H)
        if key not in cache:
            Ks = scale_K(K, w0, h0, W, H) if (w0 and h0) else K.copy()
            if not (w0 and h0):
                Ks[0,2] = W/2.0; Ks[1,2] = H/2.0

            if use_fisheye:
                map1, map2 = build_maps_fisheye(Ks, D, (W,H), balance=balance)
            else:
                map1, map2 = build_maps_standard(Ks, dist, (W,H), alpha=alpha)
            cache[key] = (map1, map2)

        map1, map2 = cache[key]
        und = cv2.remap(img, map1, map2, cv2.INTER_LINEAR)

        stem = Path(f).stem
        out_path = str(Path(out_dir) / f"{stem}_undist.jpg")
        cv2.imwrite(out_path, und, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        if side_by_side:
            sbs_path = str(Path(out_dir) / f"{stem}_compare.jpg")
            save_side_by_side(img, und, sbs_path)

def batch_undistort_with_manual(files, out_dir, manual_json, alpha=0.0, side_by_side=False):
    with open(manual_json, "r", encoding="utf-8") as f:
        data = json.load(f)
    K0   = np.array(data["K"], dtype=np.float64)
    dist = np.array(data["dist"], dtype=np.float64).reshape(-1,1)
    w0   = data.get("W", None)
    h0   = data.get("H", None)

    os.makedirs(out_dir, exist_ok=True)
    cache = {}

    for f in tqdm(files, desc="Undistorting (manual)"):
        img = cv2.imread(f)
        if img is None:
            continue
        H, W = img.shape[:2]
        key = (W,H)
        if key not in cache:
            Ks = scale_K(K0, w0, h0, W, H) if (w0 and h0) else K0.copy()
            if not (w0 and h0):
                Ks[0,2] = W/2.0; Ks[1,2] = H/2.0
            map1, map2 = build_maps_standard(Ks, dist, (W,H), alpha=alpha)
            cache[key] = (map1, map2)

        map1, map2 = cache[key]
        und = cv2.remap(img, map1, map2, cv2.INTER_LINEAR)
        stem = Path(f).stem
        out_path = str(Path(out_dir) / f"{stem}_undist.jpg")
        cv2.imwrite(out_path, und, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        if side_by_side:
            sbs_path = str(Path(out_dir) / f"{stem}_compare.jpg")
            save_side_by_side(img, und, sbs_path)


In [20]:
files = list_images(INPUT_PATH)
print(f"[INFO] 대상 이미지 수: {len(files)}")

if USE_NPZ:
    batch_undistort_with_npz(
        files, OUTPUT_DIR, CAL_NPZ_PATH,
        alpha=ALPHA, balance=BALANCE,
        side_by_side=SIDE_BY_SIDE
    )
elif USE_MANUAL_JSON:
    batch_undistort_with_manual(
        files, OUTPUT_DIR, MANUAL_JSON_PATH,
        alpha=ALPHA, side_by_side=SIDE_BY_SIDE
    )
else:
    raise SystemExit("보정 정보 없음: USE_NPZ 또는 USE_MANUAL_JSON 중 하나를 True로 설정하세요.")

print("[DONE] 저장 위치:", OUTPUT_DIR)


[INFO] 대상 이미지 수: 14


FileNotFoundError: [Errno 2] No such file or directory: 'pi_cam_calib.npz'

In [ ]:
def interactive_tune(file_path, save_json="manual_calib.json"):
    """
    Brown-Conrady(standard) 모델 간이 튜닝:
    dist=[k1,k2,p1,p2,k3], 여기서는 p1=p2=k3=0으로 두고 k1,k2만 조절.
    """
    img = cv2.imread(file_path)
    if img is None:
        raise FileNotFoundError(file_path)
    H, W = img.shape[:2]
    cx, cy = W/2.0, H/2.0

    # 초기값: 광각 가정, f는 이미지 폭 비례
    f = 0.75 * W
    K = np.array([[f,0,cx],[0,f,cy],[0,0,1.0]], dtype=np.float64)
    dist = np.zeros((5,1), dtype=np.float64)

    win = "Tune (k1,k2,f). Press S to save JSON, ESC to quit"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)

    def nothing(x): pass
    cv2.createTrackbar('k1(x1e-3)', win, -120, 200, nothing)  # -0.120 ~ +0.200
    cv2.createTrackbar('k2(x1e-3)', win, 20, 300, nothing)    # +0.020 ~ +0.300
    cv2.createTrackbar('f(%)',      win, 75, 200, nothing)    # 0.75W ~ 2.00W

    while True:
        k1 = (cv2.getTrackbarPos('k1(x1e-3)', win)) / 1000.0
        k2 = (cv2.getTrackbarPos('k2(x1e-3)', win)) / 1000.0
        f_scale = max(1e-6, cv2.getTrackbarPos('f(%)', win) / 100.0)

        K[0,0] = f_scale * W
        K[1,1] = f_scale * W
        K[0,2] = cx; K[1,2] = cy
        dist[:] = np.array([k1, k2, 0.0, 0.0, 0.0]).reshape(5,1)

        newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (W,H), 0.0)
        map1, map2 = cv2.initUndistortRectifyMap(K, dist, None, newK, (W,H), cv2.CV_16SC2)
        und = cv2.remap(img, map1, map2, cv2.INTER_LINEAR)

        comp = np.hstack([img, und])
        cv2.imshow(win, comp)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break
        if key in (ord('s'), ord('S')):
            data = {
                "model": "standard",
                "W": W, "H": H,
                "K": K.tolist(),
                "dist": dist.reshape(-1).tolist()
            }
            with open(save_json, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"[SAVED] {save_json}")
    cv2.destroyAllWindows()


In [ ]:
import piexif
from PIL import Image

def copy_exif(src_path, dst_path):
    try:
        exif_dict = piexif.load(src_path)
        exif_bytes = piexif.dump(exif_dict)
        im = Image.open(dst_path)
        im.save(dst_path, exif=exif_bytes, quality=95)
    except Exception as e:
        # EXIF 없는 경우 등
        pass

# 사용 예 (보정 후 루프에 넣어서 호출)
# for each saved file:
#     copy_exif(original_path, saved_path)
